In [1]:
import requests
from bs4 import BeautifulSoup
import pickle

In [2]:
#pip install parser-libraries

In [3]:
#pip install lxml

In [4]:
def url_to_transcript(url):
    page=requests.get(url).text
    soup=BeautifulSoup(page,"lxml")
    text=[p.text for p in soup.find(class_="elementor-element elementor-element-74af9a5b elementor-widget elementor-widget-theme-post-content").find_all('p')]
    print(url)
    return text

urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
      ]

# Comedian names
comedians = ['louis', 'dave', 'ricky']

In [5]:
transcripts = [url_to_transcript(u) for u in urls]

http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/
http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/
http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/


In [6]:
transcripts = [url_to_transcript(u) for u in urls]

http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/
http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/
http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/


In [7]:
!mkdir transcripts

for i, c in enumerate(comedians):
    with open('transcripts/' + c + '.txt', 'wb') as file:
        pickle.dump(transcripts[i], file)

A subdirectory or file transcripts already exists.


In [ ]:
data={}
for i, c in enumerate(comedians):
    with open('transcripts/' + c + '.txt', 'rb') as file:
        data[c] = pickle.load(file)

In [9]:
data.keys()

dict_keys(['louis', 'dave', 'ricky'])

In [10]:
data['louis'][:2]

['Intro Fade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily agree with you, but I appreciate very much. Well, this is a nice place. This is easily the nicest place For many miles in every direction. That’s how you compliment a building And shit on a town with one sentence. It is odd around here, as I was driving here. There doesn’t seem to be any difference Between the sidewalk and the street for pedestrians here. People just kind of walk in the middle of the road. I love traveling And seeing all the different parts of the country. I live in New York. I live in a– There’s no value to your doing that at all.',
 '“The Old Lady And The Dog” I live– I live in New York. I always– Like, there’s this old lady in my neighborhood, And she’s always walking her dog. She’s always just– she’s very old. She just stands there just being old, And the dog just fights gravity every day, just– The two of them, it’s real

## Cleaning the data

In [11]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [12]:
data_combined={key: [combine_text(value)] for (key,value) in data.items()} 

In [14]:
import pandas as pd
pd.set_option('max_colwidth', 150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns=['transcript']
data_df=data_df.sort_index()
data_df

,transcript
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
louis,Intro Fade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily ag...
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you. What a lovely welcome. I’m gonna try my hardest tonigh...


In [16]:
data_df.transcript.loc['dave']

'This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the alchemist’s fire that transforms fear and tragedy into levity and livelihood. Dave calls that look “the trance.” ♪ Play me ♪ ♪ Buy me ♪ ♪ Workinonit ♪ ♪ Tune up ♪ ♪ Tune ♪ ♪ Oh ♪  ♪ Fade me ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ ♪ In every ghetto ♪ ♪ Ah-ah, ah-ah, ah-ah ♪ Thank you! Thank you very much! Thank you all. Oh, wow. That was exciting, wasn’t it? Thank you, guys. Have a seat, feel comfortable, relax. I want to thank everyone in LA for a wonderful week. It’s been great here. You know what? It’s been ten years since the last time I played Los Angeles, if you can imagine. I know! I know, I’ve been gone for a very long time. And unb

In [17]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuiton'''
    text=text.lower()
    text=re.sub('\[.*?\]', '', text)
    text=re.sub('[%s]' %re.escape(string.punctuation), '', text)
    text=re.sub('\w*\d\w*', '', text)
    return text
round1=lambda x: clean_text_round1(x)

In [18]:
data_clean=pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
dave,this is dave he tells dirty jokes for a living that stare is where most of his hard work happens it signifies a profound train of thought the alch...
louis,intro fade the music out let’s roll hold there lights do the lights thank you thank you very much i appreciate that i don’t necessarily agree with...
ricky,hello hello how you doing great thank you wow calm down shut the fuck up thank you what a lovely welcome i’m gonna try my hardest tonight you’re t...


In [20]:
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non sensical text that was'''
    text = re.sub('\['' ""_]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [21]:
data_clean=pd.DataFrame(data_df.transcript.apply(round2))
data_clean

,transcript
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
louis,Intro Fade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily ag...
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you. What a lovely welcome. I’m gonna try my hardest tonigh...


#### corpus

In [22]:
data_df

,transcript
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ..."
louis,Intro Fade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily ag...
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you. What a lovely welcome. I’m gonna try my hardest tonigh...


In [23]:
full_names = ['Dave Chappelle', 'Louis C.K.', 'Ricky Gervais']
data_df['full_name'] = full_names
data_df

,transcript,full_name
dave,"This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought, the ...",Dave Chappelle
louis,Intro Fade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily ag...,Louis C.K.
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you. What a lovely welcome. I’m gonna try my hardest tonigh...,Ricky Gervais


In [24]:
data_df.to_pickle('corpus.pkl')

### Document-Term Matrix

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(stop_words='english')
data_cv=cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())
data_dtm.index=data_clean.index
data_dtm

,00,000,10,100,11,114,12,125,13,130,...,york,young,younger,youngest,youth,youtube,yulin,yummy,zero,zoo
dave,0,3,1,0,2,0,2,1,0,0,...,1,10,0,0,0,0,0,0,0,0
louis,1,0,1,1,0,1,0,0,0,0,...,4,8,3,0,1,1,0,0,2,0
ricky,0,4,0,2,0,0,2,0,1,1,...,0,2,0,1,0,1,1,1,0,1


In [30]:
data_dtm.to_pickle('dtm.pkl')

In [31]:
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open('cv.pkl', 'wb'))

In [32]:
from textblob import TextBlob

In [33]:
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

In [34]:
data['polarity'] = data['transcript'].apply(pol)
data['subjectivity'] = data['transcript'].apply(sub)
data

KeyError: 'transcript'